# Best params for each compartment
- for focusing on specific compartment

In [ ]:
def objective_function(params, observed_data, initial_conditions, time_points, compartment):
    if compartment == 'E':
            return E
    elif compartment == 'I':
        return I
    elif compartment == 'R':
        return R
    else:
        return E, I, R

cost_dict = dict()

observed_dict = {
    "E": train_data['count_Exposed'].to_numpy(),
    "I": train_data['count_Infectious'].to_numpy(),
    "R": train_data['count_Recovered'].to_numpy()
}
    
for compartment, observed_data in observed_dict.items():
    # Run the Simulated Annealing algorithm
    best_params, best_cost, cost_list, final_temp = simulated_annealing_seir(observed_data, initial_conditions, time_points, temp, cooling_rate, max_iter, compartment)
    o_beta, o_sigma, o_gamma = best_params

    # Display results
    print(f"Compartment: {compartment}")
    print(f"Best parameters: Beta={o_beta:.3f}, Sigma={o_sigma:.3f}, Gamma={o_gamma:.3f}")
    print(f"Best cost: {best_cost:.2f}")
    print(f"Final Temp: {final_temp:.2e}\n\n")
    
    cost_dict[compartment] = np.array(cost_list)
    
# Plot cost progression each compartment
plt.figure(figsize=(12, 6))
plt.plot(cost_dict['E'], 'r--', label='E')
plt.plot(cost_dict['I'], 'b--', label='I')
plt.plot(cost_dict['R'], 'g--', label='R')

plt.xlabel("Iteration")
plt.ylabel("R-squared (Best Cost)")
plt.title("Simulated Annealing: Progression for Each Compartment")
plt.legend(title="Compartments")
plt.grid(True)
plt.show()


# validating the best params
'''
Compartment: E
Best parameters: Beta=0.225, Sigma=0.143, Gamma=0.204
Best cost: 0.30
Final Temp: 1.05e+00


Compartment: I
Best parameters: Beta=0.241, Sigma=0.063, Gamma=0.190
Best cost: 0.29
Final Temp: 1.05e+00


Compartment: R
Best parameters: Beta=0.243, Sigma=0.042, Gamma=0.185
Best cost: 0.08
Final Temp: 1.05e+00
'''

o_beta, o_sigma, o_gamma = 0.241, 0.063, 0.190 # from best params

observed_main = df_observed_data.copy()

N = 33235

# Plot Exposed
r2_exposed = r2_score(observed_main['count_Exposed'], e)
mae_exposed = mean_absolute_error(observed_main['count_Exposed'], e)
rmse_exposed = np.sqrt(mean_squared_error(observed_main['count_Exposed'], e))
print(f"beta: {o_beta} \nr2: {r2_exposed} \nmae: {mae_exposed} \nrmse: {rmse_exposed}")

plt.figure(figsize=(12, 6))
plt.plot(plot_actual['date_train'], plot_actual['train_exposed'], 'r--', label='Train Exposed')
plt.plot(plot_actual['date_test'], plot_actual['test_exposed'], 'r-', label='Test Exposed')
plt.plot(plot_prediction['date'], plot_prediction['E'], 'ro', label='Predicted Exposed')
plt.xlabel('Days')
plt.ylabel('Population')
plt.legend()
plt.xticks(rotation=45)
plt.title(f'SEIR Model Fitting For {brgy_name} - Exposed')
plt.grid(True)
plt.show()

# Plot Infected
r2_infectious = r2_score(observed_main['count_Infectious'], i)
mae_infectious = mean_absolute_error(observed_main['count_Infectious'], i)
rmse_infectious = np.sqrt(mean_squared_error(observed_main['count_Infectious'], i))
print(f"sigma: {o_sigma} \nr2: {r2_infectious} \nmae: {mae_infectious} \nrmse: {rmse_infectious}")

plt.figure(figsize=(12, 6))
plt.plot(plot_actual['date_train'], plot_actual['train_infectious'], 'b--', label='Train Infectious')
plt.plot(plot_actual['date_test'], plot_actual['test_infectious'], 'b-', label='Test Infectious')
plt.plot(plot_prediction['date'], plot_prediction['I'], 'bo', label='Predicted Infected')
plt.ylabel('Population')
plt.legend()
plt.xticks(rotation=45)
plt.title(f'SEIR Model Fitting For {brgy_name} - Infected')
plt.grid(True)
plt.show()

# Plot Recovered
r2_recovered = r2_score(observed_main['count_Recovered'], r)
mae_recovered = mean_absolute_error(observed_main['count_Recovered'], r)
rmse_recovered = np.sqrt(mean_squared_error(observed_main['count_Recovered'], r))
print(f"gamma: {o_gamma} \nr2: {r2_recovered} \nmae: {mae_recovered} \nrmse: {rmse_recovered}")

plt.figure(figsize=(12, 6))
plt.plot(plot_actual['date_train'], plot_actual['train_recovered'], 'g--', label='Train Recovered')
plt.plot(plot_actual['date_test'], plot_actual['test_recovered'], 'g-', label='Test Recovered')
plt.plot(plot_prediction['date'], plot_prediction['R'], 'go', label='Predicted Recovered')
plt.xlabel('Days')
plt.ylabel('Population')
plt.legend()
plt.xticks(rotation=45)
plt.title(f'SEIR Model Fitting For {brgy_name} - Recovered')
plt.grid(True)
plt.show()

In [ ]:
# Objective function
def objective_function(params, observed_data, initial_conditions, time_points, n):
    beta, sigma, gamma = params

    # Simulate the SEIR model with given parameters
    s, predicted_E, predicted_I, predicted_R = simulate_seir(beta, sigma, gamma, initial_conditions, time_points, n)
    
    # Split observed data into compartments
    observed_E, observed_I, observed_R = observed_data[:, 0], observed_data[:, 1], observed_data[:, 2]
    
    # Calculate metrics for each compartment
    metrics = {}
    r2_dict= {}
    mae_dict = {}
    rmse_dict = {}
    
    for compartment, observed, predicted in zip(
        ["E", "I", "R"],
        [observed_E, observed_I],
        [predicted_E, predicted_I]
        ):
        # [observed_E, observed_I, observed_R],
        # [predicted_E, predicted_I, predicted_R]
        # ):
    
        r2 = r2_score(observed, predicted)
        mae = mean_absolute_error(observed, predicted)
        rmse = np.sqrt(mean_squared_error(observed, predicted))

        # Normalize metrics
        r2_normalized = 1 - r2  # Invert R^2 for minimization
        mae_normalized = mae / np.max(observed)  # Scale by max value
        rmse_normalized = rmse / np.max(observed)  # Scale by max value

        # Weighted average for each compartment
        combined_metric = (0.2 * r2_normalized) + (0.4 * mae_normalized) + (0.4 * rmse_normalized)
        metrics[compartment] = combined_metric
        r2_dict[compartment] = r2
        mae_dict[compartment] = mae
        rmse_dict[compartment] = rmse

    # Combine all compartments (average or weighted sum)
    total_metric = np.mean(list(metrics.values()))  # Equal weight to all compartments
    total_r2 = np.mean(list(r2_dict.values()))
    total_mae = np.mean(list(mae_dict.values()))
    total_rmse = np.mean(list(rmse_dict.values()))
    
    return total_metric, total_r2, total_mae, total_rmse